In [94]:
# 1. Import necessary libraries
import pandas as pd
import os
import math

In [95]:
# google analytics data cleanup

class GoogleAnalyticsDataProcessing():
    def __init__(self, council_name, fiscal_year) -> None:
        self.suffixes_to_remove = ["?fbclid=", "+&", "?_x_tr_", "?back="]
        self.search_cache_identifier = "/search?q=cache:"
        self.sacommunity_url = "https://sacommunity.org"
        self.council_name = council_name
        self.fiscal_year = fiscal_year
        self.data_root_folder = './data'
        self.google_analytics_landing_page_df = None
        self.sa_community_data_gov_au_export_df = None

    def get_data_folder_path(self) -> str:
        return os.path.join(self.data_root_folder, self.council_name, self.fiscal_year)

    def read_google_analytics_landing_page_data(self, file_name: str) -> pd.DataFrame:
        full_file_name = os.path.join(self.get_data_folder_path(), file_name)
        self.google_analytics_landing_page_df = pd.read_excel(full_file_name, sheet_name='Dataset1')
        return self.google_analytics_landing_page_df

    def read_sacommunity_data_gov_au_export(self, file_name: str) -> pd.DataFrame:
        full_file_name = os.path.join(self.get_data_folder_path(), file_name)
        self.sa_community_data_gov_au_export_df = pd.read_csv(full_file_name)
        return self.sa_community_data_gov_au_export_df

    def save_processed_data(self, data_df : pd.DataFrame, file_name: str):
        full_file_name = os.path.join(self.get_data_folder_path(), file_name)
        data_df.to_csv(full_file_name, index=False)

    def clean_landing_page(self, text: str) -> str:
        if self.search_cache_identifier in text:
            text = text[text.index(self.sacommunity_url):].replace(self.sacommunity_url, "")

        for suffix_to_remove in self.suffixes_to_remove:
            if suffix_to_remove in text:
                text = text[:text.index(suffix_to_remove)]

        # remove underscore
        text = text.replace("_", " ")
        # remove /org/
        text = text.replace("/org/", "")

        return text.strip()

    def get_organization_id(self, text: str) -> str:
        if "-" in text:
            return int(text[:text.index("-")])
        else:
            return None
    
    def get_organization_name(self, text: str) -> str:
        if "-" in text:
            return text[text.index("-") + 1:]
        else:
            return None
        
    def get_sessions_by_organization(self, df_ga_orig: pd.DataFrame) -> pd.DataFrame:
        df_ga = df_ga_orig.dropna().copy()
        df_ga['organization_id_name'] = df_ga['Landing Page'].apply(self.clean_landing_page)
        df_ga['organization_id'] = df_ga['organization_id_name'].apply(self.get_organization_id)
        df_ga['organization_name'] = df_ga['organization_id_name'].apply(self.get_organization_name)
        return df_ga[["Landing Page", "organization_id_name", "organization_id", "organization_name", "Sessions"]]
    
    def group_sessions_by_organization(self, google_analytics_cleaned_df) -> pd.DataFrame:
        return google_analytics_cleaned_df.groupby(by=['organization_id']).sum("Sessions")

    def process_data(self, google_analytics_sessions_data_df, sa_community_df) -> pd.DataFrame:
        results = []
        for _, row in google_analytics_sessions_data_df.iterrows():
            org_id_str = row["organization_id"]
            if math.isnan(org_id_str):
                print('org id is invalid, so skip it ', org_id_str)
                continue
            org_id = 0
            if org_id_str is not None:
                org_id = int(org_id_str)
            
            session_count = row["Sessions"]
            
            # organization name from sa-community file
            org_names_sa_community = sa_community_df[sa_community_df['ID_19'] == org_id]["Org_name"].values
            organization_name_sa_community = ''
            is_record_available_in_sacommunity_db = False
            if len(org_names_sa_community) > 0:
                organization_name_sa_community = org_names_sa_community[0]
                is_record_available_in_sacommunity_db = True
        
            # organization name from google analytics file
            org_names_google = google_analytics_sessions_data_df[google_analytics_sessions_data_df["organization_id"] == org_id]["organization_name"].values
            # print('org_names_google ', org_names_google)
            organization_name_google = ''
            if len(org_names_google) > 0:
                organization_name_google = org_names_google[0]

            # print('org_names_google ', org_names_google)
            landing_page = self.sacommunity_url + google_analytics_sessions_data_df[google_analytics_sessions_data_df["organization_id"] == org_id]["Landing Page"].values[0]
            results.append({
                'org_id': org_id,
                'landing_page': landing_page,
                'sessions_count': session_count,
                'organization_name_sa_community': organization_name_sa_community,
                'organization_name_google': organization_name_google,
                'is_record_available_in_sacommunity_db': is_record_available_in_sacommunity_db,
            })

        return pd.DataFrame(results)

# test texts
# google_analytics_processing = GoogleAnalyticsDataProcessing()
# inputs = [
#     "/org/196236-Dave's_Angels_Playgroup?fbclid=IwAR05WAQ0z5mwY7v1UEVmkDITFg7sDh8pcD8taJ3oGH4336EpkNZeP81BIKc",
#     "/search?q=cache:UTs_a-1ZNgEJ:https://sacommunity.org/org/196341-Neighbourhood_Watch_-_Linden_Park_249+&cd=63&hl=en&ct=clnk&gl=bj",
#     "/org/201669-Gifted_&_Talented_Children's_Association_of_SA_Inc.?_x_tr_sl=en&_x_tr_tl=th&_x_tr_hl=th&_x_tr_pto=sc",
#     "/org/201830-Aged_Rights_Advocacy_Service_Inc.?back=https://www.google.com/search?client=safari&as_qdr=all&as_occt=any&safe=active&as_q=Age+advocate+for+South+Australia&channel=aplab&source=a-app1&hl=en",
#     "/org/201950-SA_Ambulance_Service?_x_tr_sl=en&_x_tr_tl=fr&_x_tr_hl=fr&_x_tr_pto=nui,sc"
# ]

# for input in inputs:
#     print(google_analytics_processing.clean_landing_page(input))
    

In [96]:
COUNCIL = 'Burnside'
FISCAL_YEAR = '2022-2023'
data_processing = GoogleAnalyticsDataProcessing(COUNCIL, FISCAL_YEAR)


In [97]:
# google analytics data
google_analytics_landing_page_df = data_processing.read_google_analytics_landing_page_data('google_analytics_landing_page.xlsx')
google_analytics_landing_page_df.head()

/Users/pujanmaharjan/anaconda/anaconda3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Custom Variable (Value 01),Landing Page,Sessions,% New Sessions,New Users,Bounce Rate,Pages / Session,Avg. Session Duration,Goal Conversion Rate,Goal Completions,Goal Value
0,0QK91R12,/org/201829-Cats_Assistance_To_Sterilise_Inc.,514,0.859922,442,0.809339,1.161479,41.268482,0,0,0
1,0QK91R12,/org/201950-SA_Ambulance_Service,319,0.884013,282,0.793103,1.150470,20.442006,0,0,0
2,0QK91R12,/org/201612-Dementia_Australia,271,0.797048,216,0.892989,1.077491,36.151292,0,0,0
3,0QK91R12,/org/202703-Drug_and_Alcohol_Services_SA_-_Wit...,271,0.782288,212,0.819188,1.140221,38.811808,0,0,0
4,0QK91R12,/org/236722-Justices_of_the_Peace_-_Burnside,265,0.875472,232,0.867925,1.079245,24.573585,0,0,0


In [98]:
# SAcommunity data
sa_community_data_gov_au_export_df = data_processing.read_sacommunity_data_gov_au_export('sacommunity_data_gov_au_export.csv')
sa_community_data_gov_au_export_df.head()

,ID_19,Org_name,Street_Address_Line_1,Street_Address_Line_2,Suburb,State,Postal_Code,Country,Postal_Address_Line_1,Postal_Address_Line_2,...,Organisation_Last_updated,IM_Screen_Name_1,IM_Screen_Name_2,IM_Screen_Name_3,IM_Screen_Name_4,IM_Screen_Name_5,IM_Screen_Name_6,Organisati_Council,Organisati_Electorate_State_,Organisati_Electorate_Federal_
0,194023,Motorcycling South Australia Inc.,"Motorcycling South Australia Inc., 251 The Pde",NaN,Beulah Park,South Australia,5067.0,Australia,NaN,NaN,...,2022-11-16 11:54:08,https://www.facebook.com/motorcyclingsa/ 7,NaN,NaN,NaN,NaN,NaN,City of Burnside,Dunstan,Sturt
1,194025,Orienteering SA,NaN,NaN,Glenside,South Australia,5065.0,Australia,c/o 5/355 Angas St,NaN,...,2022-07-24 15:14:31,https://www.facebook.com/OrienteeringSA 7,https://www.youtube.com/channel/UCSrZVpB1et3Jo...,NaN,NaN,NaN,NaN,City of Burnside,Bragg,Sturt
2,194813,Burnside Library & Information Service,401 Greenhill Rd,NaN,Tusmore,South Australia,5065.0,Australia,PO Box 9,NaN,...,2022-12-07 10:38:10,BurnsideLibrary 7,BurnsideLibrary 4,burnsidelibrary 1,NaN,NaN,NaN,City of Burnside,Bragg,Sturt
3,196167,Burnside Family Church,88 Lockwood Rd,NaN,Burnside,South Australia,5066.0,Australia,NaN,NaN,...,2023-01-25 13:53:04,burnsidefamilychurch 7,BFamilyChurch 4,NaN,NaN,NaN,NaN,City of Burnside,Bragg,Sturt
4,196171,Linden Park Primary School and OSHC/VAC,"Linden Park Primary School, 14 Hay Rd",NaN,Linden Park,South Australia,5065.0,Australia,NaN,NaN,...,2023-01-25 14:06:43,https://www.facebook.com/groups/3219860600/abo...,NaN,NaN,NaN,NaN,NaN,City of Burnside,Bragg,Sturt


In [99]:
sessions_data_df = data_processing.get_sessions_by_organization(google_analytics_landing_page_df)
processed_data_df = data_processing.process_data(sessions_data_df, sa_community_data_gov_au_export_df)
processed_data_df.head()

org id is invalid, so skip it  nan


,org_id,landing_page,sessions_count,organization_name_sa_community,organization_name_google,is_record_available_in_sacommunity_db
0,201829,https://sacommunity.org/org/201829-Cats_Assist...,514,Cats Assistance To Sterilise Inc.,Cats Assistance To Sterilise Inc.,True
1,201950,https://sacommunity.org/org/201950-SA_Ambulanc...,319,SA Ambulance Service,SA Ambulance Service,True
2,201612,https://sacommunity.org/org/201612-Dementia_Au...,271,Dementia Australia,Dementia Australia,True
3,202703,https://sacommunity.org/org/202703-Drug_and_Al...,271,Drug and Alcohol Services SA - Withdrawal Serv...,Drug and Alcohol Services SA - Withdrawal Serv...,True
4,236722,https://sacommunity.org/org/236722-Justices_of...,265,Justices of the Peace - Burnside,Justices of the Peace - Burnside,True


In [110]:
data_processing.save_processed_data(processed_data_df, 'processed_data.csv')

In [112]:
# these records are problematic, they are found in google analytics, but not in sacommunity council based export 
# Check these records manually, why it is not available in sacommunity db
# One posible reason is that the record in sacommunity is invalid, the council name could be wrong
# Later, will try to automate on how to get the exact council name from selenium
data_available_in_google_analytics_but_not_in_sacommunity_df = processed_data_df[processed_data_df["is_record_available_in_sacommunity_db"] == False]
data_available_in_google_analytics_but_not_in_sacommunity_df.head()

,org_id,landing_page,sessions_count,organization_name_sa_community,organization_name_google,is_record_available_in_sacommunity_db
6,208832,https://sacommunity.org/org/208832-Burnside_Yo...,209,,Burnside Youth Club,False
15,196519,https://sacommunity.org/org/196519-Sturt_Badmi...,108,,Sturt Badminton Club Inc.,False
18,196343,https://sacommunity.org/org/196343-Gospel_Pres...,95,,Gospel Presbyterian Church,False
22,214815,https://sacommunity.org/org/214815-Sikh_Societ...,83,,Sikh Society of SA,False
53,196237,https://sacommunity.org/org/196237-Australian_...,36,,Australian Girls Choir,False


In [113]:

data_available_in_google_analytics_but_not_in_sacommunity_full_filename = os.path.join(data_processing.get_data_folder_path(), 'data_available_in_google_analytics_but_not_in_sacommunity.csv')
data_available_in_google_analytics_but_not_in_sacommunity_df.to_csv(data_available_in_google_analytics_but_not_in_sacommunity_full_filename, index=False)

In [108]:
# Sample code on how selenium works
# from selenium import webdriver
# from selenium.webdriver.common.by import By

# from selenium import webdriver
# from chromedriver_py import binary_path # this will get you the path variable


# # # deprecated but works in older selenium versions
# # # driver = webdriver.Chrome(executable_path=binary_path)
# # driver.get("http://www.python.org")
# # assert "Python" in driver.title

# def test_eight_components():
#     driver = webdriver.Chrome(executable_path=binary_path)

#     driver.get("https://www.selenium.dev/selenium/web/web-form.html")

#     title = driver.title
#     assert title == "Web form"

#     driver.implicitly_wait(10) # 0.5

#     text_box = driver.find_element(by=By.NAME, value="my-text")
#     submit_button = driver.find_element(by=By.CSS_SELECTOR, value="button")

#     text_box.send_keys("Selenium")
#     submit_button.click()

#     message = driver.find_element(by=By.ID, value="message")
#     value = message.text
#     assert value == "Received!"

#     driver.quit()

# test_eight_components()

In [109]:
# Attempt to retrieve data from arcgis website, "https://lga-sa.maps.arcgis.com/apps/instant/lookup/index.html?appid=db6cce7b773746b4a1d4ce544435f9da&find=130%20L%27Estrange%20Street%2C%20Glenunga"
# TODO: this doesnot work at the moment, will try again with some other approach
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from chromedriver_py import binary_path # this will get you the path variable

# async def get_text_by_xpath(driver, x_path):
#     return await driver.find_element(by=By.CSS_SELECTOR, value=x_path).text

# driver = webdriver.Chrome(executable_path=binary_path)

# driver.get("https://lga-sa.maps.arcgis.com/apps/instant/lookup/index.html?appid=db6cce7b773746b4a1d4ce544435f9da&find=130%20L%27Estrange%20Street%2C%20Glenunga")

# # driver.implicitly_wait(60)
# print('page source ', driver.page_source)
# # //*[@id="FindMyCouncil_944448"]/div/div/div/div/div/div/table/tbody/tr[1]/th
# # h1 = driver.find_element(by=By.XPATH, value='//*[@id="FindMyCouncil_944448"]/div/div/div/div/div/div/table/tbody/tr[1]/th')
# # v1 = driver.find_element(by=By.XPATH, value='//*[@id="FindMyCouncil_944448"]/div/div/div/div/div/div/table/tbody/tr[1]/td')
# # h2 = driver.find_element(by=By.XPATH, value='//*[@id="FindMyCouncil_944448"]/div/div/div/div/div/div/table/tbody/tr[2]/th')
# # v2 = driver.find_element(by=By.XPATH, value='//*[@id="FindMyCouncil_944448"]/div/div/div/div/div/div/table/tbody/tr[2]/td')
# # h3 = driver.find_element(by=By.XPATH, value='//*[@id="FindMyCouncil_944448"]/div/div/div/div/div/div/table/tbody/tr[3]/th')
# # v3 = driver.find_element(by=By.XPATH, value='//*[@id="FindMyCouncil_944448"]/div/div/div/div/div/div/table/tbody/tr[3]/td')
# print('getting element for council')
# print(await get_text_by_xpath(driver, '#FindMyCouncil_944448 > div > div > div > div > div > div > table > tbody > tr:nth-child(1) > th'))
# print()
# # print(v1.text)

# driver.quit